In [19]:
from flask import Flask
from flask import request, jsonify
from flask_restful import Resource, Api, reqparse
import pandas as pd
import ast
import psycopg2
import pandas.io.sql as sqlio
import requests
import sqlalchemy
from sqlalchemy import create_engine
from pathlib import Path
import json
from multipledispatch import dispatch
import os

# Helper Functions

In [20]:
def get_properties():
    props = {}
    separator = "="
            
    with open('./database.properties') as file:
        for line in file: 
            if separator in line:
                name, value = line.split(separator, 1)
                props[name.strip()] = value.strip()
    return props

# Databaseclass Database:

In [21]:
class Database:
    def __init__(self):
        db_prop = get_properties()
        self.username = db_prop.get('username')
        self.password = db_prop.get('password')
        self.host = db_prop.get('host')
        self.port = db_prop.get('port')
        self.database = db_prop.get('database')
        #For SQLAlchemy
       
    
    def connect(self):
        connection = psycopg2.connect(user=self.username,
                                      password=self.password,
                                      host=self.host,
                                      port=self.port,
                                      database=self.database)
        return connection, connection.cursor()
    
    def connect_alchemy(self):
        '''Alt library to connect used for importing dataframes'''
        engine = create_engine('postgresql://'+self.username+':'+self.password+'@'+self.host+':'+self.port+'/'+self.database)
        return engine
    
    @dispatch(str)
    def insert_habit(self, name):
        query = 'INSERT INTO public.task ("name") VALUES (%s);'
        values = tuple(name)
        conn, cursor = None, None
        response = ""
        try:
            conn, cursor = self.connect()
            cursor.execute(query, (name,))
            conn.commit()
            query = "select * from public.task WHERE name = %s and date =%s"
            cursor.execute(query, values)
            conn.commit()
            response = cursor.fetchone()[0]
        except (Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
            response = "duplicate"
        finally:
            # closing database connection.
            if conn:
                cursor.close()
                conn.close()
            return response
    @dispatch(str, str)            
    def insert_habit(self, name, date):
        query = 'INSERT INTO public.task ("name", "date") VALUES (%s, %s);'
        values = (name, date,)
        conn, cursor, task_id = None, None, None
        response = ""
        try:
            conn, cursor = self.connect()
            cursor.execute(query, values)
            conn.commit()
            query = "select * from public.task WHERE name = %s and date =%s"
            cursor.execute(query, values)
            conn.commit()
            response = cursor.fetchone()[0]
        except (Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
            response = "duplicate"
        finally:
            # closing database connection.
            if conn:
                cursor.close()
                conn.close()
            return response
        
    def read_habits(self):
        query = "select * from task"
        result = None
        conn, cursor = None, None
        try:
            conn, cursor = self.connect()
            result = sqlio.read_sql_query(query, conn)
        except(Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
        finally:
            if conn:
                cursor.close()
                conn.close()
            return result     

# Make API

In [22]:
app = Flask(__name__)
api = Api(app)

In [23]:
class Habit(Resource):
    def __init__(self):
        self.db = Database()
    
    def get(self):
        return {"test":"Woo"}
    def post(self):
        parser = reqparse.RequestParser()
        parser.add_argument('habitName', required = True)
        parser.add_argument('date', required = True)
        args = parser.parse_args()
        if "today" in args["date"]:
            result = self.db.insert_habit(args["habitName"])
        else: 
            result = self.db.insert_habit(args["habitName"], args["date"])
        print("Inserting " + args["habitName"] + "  " +args["date"])
        print(result)
        if result is "duplicate":
            return jsonify(isError= True,
                    message= "Duplicate",
                    statusCode= 200,
                    data= args)
        return jsonify(isError= False,
                    message= "Success",
                    statusCode= 200,
                    data= args)
    
api.add_resource(Habit, '/habit')

In [26]:
if __name__ == '__main__':
    app.run(host="0.0.0.0")

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.86.25:5000/ (Press CTRL+C to quit)
192.168.86.25 - - [14/Dec/2021 11:52:15] "POST //habit HTTP/1.1" 200 -


Inserting test  2021-10-01
1760
